# 第4章 NumPy基础:数组与向量化计算

使用astype时总是生成一个新的数组,即使你传入的dtype与之前一样。
```python
a = np.array([1, 2, 3])
b = a.astype('uint8')
```

区别于Python的内建列表,numpy的切片是原数组的视图。这意味着数据并不是被复制了,任何对于视图的修改都会反映到原数组上。
```python
import numpy as np

arr = np.arange(10)
print('arr:', arr)
arr_slice = arr[5:8]
print('arr_slice:', arr_slice)
arr_slice[1] = 12345
print('arr:', arr)
arr_slice[:] = 64
print('arr:', arr)

>>>
arr: [0 1 2 3 4 5 6 7 8 9]
arr_slice: [5 6 7]
arr: [    0     1     2     3     4     5 12345     7     8     9]
arr: [ 0  1  2  3  4 64 64 64  8  9]

```

numpy显式复制数组为：
```python
arr[5:8].copy()
```

可以通过传递一个索引的逗号分隔列表去选择单个元素,以下两种方式效果一样:
```python
>>> arr2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
>>> arr2d[0][2]
>>> 3
>>> arr2d[0, 2]
>>> 3
```
我们可以将0轴看作“行”,将1轴看作“列”

Python的关键字and和or对numpy布尔值数组并没有用,请使用&(and)和|(or)来代替。
使用布尔值索引选择数据时,总是生成数据的拷贝,即使返回的数组并没有任何变化
```python
mask = (names == 'Bob') | (names == 'Will')
```

## 4.1.6 神奇索引

神奇索引是NumPy中的术语,用于描述使用整数数组进行数据索引。
```python
>>> arr = np.empty((8, 4))
>>> for i in range(8):
        arr[i] = i
>>> array([[0., 0., 0., 0.],
           [1., 1., 1., 1.],
           [2., 2., 2., 2.],
           [3., 3., 3., 3.],
           [4., 4., 4., 4.],
           [5., 5., 5., 5.],
           [6., 6., 6., 6.],
           [7., 7., 7., 7.]])
```
为了选出一个符合特定顺序的子集,你可以简单地通过传递一个包含指明所需顺序的列表或数组来完成:
```python
>>> arr[[4, 3, 0, 6]]
>>> array([[4., 4., 4., 4.],
           [3., 3., 3., 3.],
           [0., 0., 0., 0.],
           [6., 6., 6., 6.]])
```
如果使用负的索引,将从尾部进行选择:
```python
>>> arr[[-3, -5, -7]]
>>> array([[5., 5., 5., 5.],
           [3., 3., 3., 3.],
           [1., 1., 1., 1.]])
```
传递多个索引数组时情况有些许不同,这样会根据每个索引元组对应的元素选出一个一维数组:
```python
>>> arr = np.arange(32).reshape((8, 4))
>>> arr
>>> array([[ 0,  1,  2,  3],
           [ 4,  5,  6,  7],
           [ 8,  9, 10, 11],
           [12, 13, 14, 15],
           [16, 17, 18, 19],
           [20, 21, 22, 23],
           [24, 25, 26, 27],
           [28, 29, 30, 31]])
>>> arr[[1, 5, 7, 2], [0, 3, 1, 2]]
>>> array([ 4, 23, 29, 10])
```

# 4.3.1 将条件逻辑作为数组操作

numpy.where函数是三元表达式x if condition else y的向量化版本。假设我们有一个布尔值数组和两个数值数组:
```python
xarr = np.array([1.1, 1.2, 1.3, 1.4, 1.5])
yarr = np.array([2.1, 2.2, 2.3, 2.4, 2.5])
cond = np.array([True, False, True, True, False])
# 假设cond中的元素为True时,我们取xarr中的对应元素值,否则取yarr中的元素。
# 我们可以通过列表推导式来完成,像下列代码这样
result = [(x if c else y) for x, y, c in zip(xarr, yarr, cond)]
# 这样会产生多个问题。首先,如果数组很大的话,速度会很慢(因为所有的工作都是通过解释器解释Python代码完成)。
# 其次,当数组是多维时,就无法凑效了。而使用np.where时,就可以非常简单地完成
result = np.where(cond, xarr, yarr)
# np.where的第二个和第三个参数并不需要是数组,它们可以是标量。
# where在数据分析中的一个典型用法是根据一个数组来生成一个新的数组
```